# Data preprocessing

## Check the dataset

Import the packages

In [92]:
import pandas as pd
import numpy as np

In [93]:
df=pd.read_csv('../OriginalDataset/data_weather.csv')

In [94]:
df.head(3)

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,19790102,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,19790103,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0


In [95]:
df.tail(3)

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
15338,20201229,7.0,0.0,21.0,4.1,2.6,1.1,0.0,98830.0,NaN
15339,20201230,6.0,0.4,22.0,5.6,2.7,-0.1,0.0,100200.0,NaN
15340,20201231,7.0,1.3,34.0,1.5,-0.8,-3.1,0.0,100500.0,NaN


In [96]:
print(df.dtypes)

date                  int64
cloud_cover         float64
sunshine            float64
global_radiation    float64
max_temp            float64
mean_temp           float64
min_temp            float64
precipitation       float64
pressure            float64
snow_depth          float64
dtype: object


Find the number of null values for each feature

In [97]:
null_values=df.isnull().sum()
print(null_values)

date                   0
cloud_cover           19
sunshine               0
global_radiation      19
max_temp               6
mean_temp             36
min_temp               2
precipitation          6
pressure               4
snow_depth          1441
dtype: int64


Convert the 'date' to date type

In [98]:
df['date']=pd.to_datetime(df['date'], format='%Y%m%d')
df['date'].head(3)

0   1979-01-01
1   1979-01-02
2   1979-01-03
Name: date, dtype: datetime64[ns]

## Data cleaning (process the outliers)

Use 3-sigma principle to find the outliers

In [99]:
# Calculate the mean value and standard deviation of mean_temp and precipitation
# mean_temp
mean_temp_mean=df['mean_temp'].mean()
mean_temp_std=df['mean_temp'].std()

# precipitation
precipitation_mean=df['precipitation'].mean()
precipitation_std=df['precipitation'].std()

# max_temp
max_temp_mean=df['mean_temp'].mean()
max_temp_std=df['mean_temp'].std()

# min_temp
min_temp_mean=df['mean_temp'].mean()
min_temp_std=df['mean_temp'].std()

Use 3-sigma principle to find the normal boundary of mean_temp and precipitation in the dataset

In [100]:
# mean_temp
mean_temp_lower_bound=mean_temp_mean-3*mean_temp_std
mean_temp_upper_bound=mean_temp_mean+3*mean_temp_std

# precipitation
precipitation_lower_bound=precipitation_mean-3*precipitation_std
precipitation_upper_bound=precipitation_mean+3*precipitation_std

# max_temp
max_temp_lower_bound=mean_temp_mean-3*mean_temp_std
max_temp_upper_bound=mean_temp_mean+3*mean_temp_std

# min_temp
min_temp_lower_bound=mean_temp_mean-3*mean_temp_std
min_temp_upper_bound=mean_temp_mean+3*mean_temp_std

Calculate the number of the outliers of mean_temp and precipitation

In [101]:
# find the outliers
mean_temp_outliers=df[(df['mean_temp']<mean_temp_lower_bound) | (df['mean_temp']>mean_temp_upper_bound)]
precipitation_outliers=df[(df['precipitation']<precipitation_lower_bound) | (df['precipitation']>precipitation_upper_bound)]
max_temp_outliers=df[(df['max_temp']<max_temp_lower_bound) | (df['max_temp']>max_temp_upper_bound)]
min_temp_outliers=df[(df['min_temp']<min_temp_lower_bound) | (df['min_temp']>min_temp_upper_bound)]

# calculate the number of outliers
num_mean_temp_outliers=len(mean_temp_outliers)
num_precipitation_outliers=len(precipitation_outliers)
num_max_temp_outliers=len(max_temp_outliers)
num_min_temp_outliers=len(min_temp_outliers)

print("The number of outliers in mean_temp：",num_mean_temp_outliers)
print("The number of outliers in precipitation：",num_precipitation_outliers)
print("The number of outliers in max_temp：",num_max_temp_outliers)
print("The number of outliers in min_temp：",num_min_temp_outliers)

The number of outliers in mean_temp： 5
The number of outliers in precipitation： 337
The number of outliers in max_temp： 289
The number of outliers in min_temp： 52


In [102]:
# Extract the dates of mean_temp outliers
dates_of_mean_temp_outliers = mean_temp_outliers[['date','mean_temp']]
dates_of_mean_temp_outliers

,date,mean_temp
2933,1987-01-12,-7.6
2934,1987-01-13,-6.2
8987,2003-08-10,29.0
13330,2015-07-01,28.7
14815,2019-07-25,28.8


Delete the outliers in mean_temp and precipitation

In [103]:
df.loc[(df['mean_temp']<mean_temp_lower_bound) | (df['mean_temp']>mean_temp_upper_bound),'mean_temp']=np.nan
df.loc[(df['max_temp']<max_temp_lower_bound) | (df['max_temp']>max_temp_upper_bound),'max_temp']=np.nan
df.loc[(df['min_temp']<min_temp_lower_bound) | (df['min_temp']>min_temp_upper_bound),'min_temp']=np.nan
df.loc[(df['precipitation']< precipitation_lower_bound) | (df['precipitation']>precipitation_upper_bound),'precipitation']=np.nan

## Data imputation

### Impute cloud_cover

In [104]:
# Impute NaN in cloud_cover with linear method
df['cloud_cover']=df['cloud_cover'].interpolate(method='linear',limit_direction='both')
df['cloud_cover']=df['cloud_cover'].round(1)

print(df['cloud_cover'].isnull().sum())

0


### Impute global_radiation

In [105]:
# Impute NaN in global_radiation with linear method
df['global_radiation']=df['global_radiation'].interpolate(method='linear', limit_direction='both')
df['global_radiation']=df['global_radiation'].round(1)

print(df['global_radiation'].isnull().sum())


0


### Impute precipitation

In [106]:
# Impute NaN in precipitation with linear method
df['precipitation']=df['precipitation'].interpolate(method='linear', limit_direction='both')
df['precipitation']=df['precipitation'].round(1)

print(df['precipitation'].isnull().sum())

0


### Impute pressure

In [107]:
# Impute NaN in pressure with linear method
df['pressure']=df['pressure'].interpolate(method='linear', limit_direction='both')
df['pressure']=df['pressure'].round(1)

print(df['pressure'].isnull().sum())

0


### Impute temperature

Impute min_temp

In [108]:
# Impute NaN in global_radiation with linear method
df['min_temp']=df['min_temp'].interpolate(method='linear', limit_direction='both')
df['min_temp']=df['min_temp'].round(1)

print(df['min_temp'].isnull().sum())

0


Impute max_temp

In [109]:
# Impute NaN in global_radiation with linear method
df['max_temp']=df['max_temp'].interpolate(method='linear', limit_direction='both')
df['max_temp']=df['max_temp'].round(1)

print(df['max_temp'].isnull().sum())

0


Impute mean_temp

In [110]:
# Find the rows whose value of mean_temp is NaN
null_rows=df[df['mean_temp'].isnull()]

# Calculate the mean value of min_temp and max_temp
for index, row in null_rows.iterrows():
    min_temp=row['min_temp']
    max_temp=row['max_temp']
    mean_temp=np.mean([min_temp, max_temp])
    df.at[index,'mean_temp']=mean_temp

df['mean_temp']=df['mean_temp'].round(1)
print(df['mean_temp'].isnull().sum())

0


### Impute snow_depth

Calculate the number of days whose snow_depth is not zero for each month

In [111]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month

snow_days_notzero=df[df['snow_depth'] != 0].groupby(['month'])['snow_depth'].count()
print(snow_days_notzero)

month
1     56
2     48
3      6
4      2
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12    28
Name: snow_depth, dtype: int64


We can learn that from May to November, there are no days that have snow_depth and there are only 2 days with snow_depth in April and 6 days in March with forty years. So impute NaN of snow_depth to 0 in these months as follows.

In [112]:
# Impute NaN of snow_depth to 0
months=[3,4,5,6,7,8,9,10,11]
df.loc[(df['month'].isin(months))&(df['snow_depth'].isnull()),'snow_depth']=0

Impute the NaN in snow_depth before 2020/1/1 using linear method

In [113]:
# Define the start date
start_date = pd.to_datetime('2020-01-01')

# Using linear method to impute NaN value in snow_depth before the start date
df.loc[df['date'] < start_date, 'snow_depth'] = df.loc[df['date'] < start_date, 'snow_depth'].interpolate(method='linear')


Impute the NaN value in snow_depth after 2020/1/1 (Using support vector machine model to predict the value of snow_depth)

In [114]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X=df.drop(columns=['date','snow_depth','month','year'])
Y=df['snow_depth'].apply(lambda x: 1 if x > 0 else 0)

# Set the proportion of training set to 0.8.
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,shuffle=False)

svm_classifier = SVC()
svm_classifier.fit(X_train,Y_train)

# Predict the values of NaN in snow_depth
months_to_fill=[1, 2, 12]
data_to_fill=df[df['snow_depth'].isnull()&df['month'].isin(months_to_fill)&df['year'].isin([2020])]
X_to_fill=data_to_fill.drop(columns=['snow_depth','date','year','month'])
predicted_values=svm_classifier.predict(X_to_fill)

# Impute the NaN in snow_depth using the predicted values
df.loc[df['snow_depth'].isnull()&df['date'].dt.month.isin(months_to_fill),'snow_depth']=predicted_values


### Check the NaN values in each feature after imputing

In [115]:
# write the new dataset to the csv file
df.to_csv('../dataset_for_analysis/data_weather_new1.csv',index=0)

In [116]:
df_new=pd.read_csv('../dataset_for_analysis/data_weather_new1.csv')
null_values=df_new.isnull().sum()
print(null_values)

date                0
cloud_cover         0
sunshine            0
global_radiation    0
max_temp            0
mean_temp           0
min_temp            0
precipitation       0
pressure            0
snow_depth          0
year                0
month               0
dtype: int64
